In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to buy']
def stock_list(values):
    df = pd.DataFrame(columns=my_columns)
        
    for name in stocks['Ticker'][:1]:
        api_url = f'https://sandbox.iexapis.com/stable/stock/{name}/quote?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
    
        df = df.append(
            pd.Series(
                [
                    name,
                    data['latestPrice'],
                    data['marketCap'],
                    'N/A'
                    ],
                    index=my_columns
                    ),
            ignore_index=True
            )
    # df = pd.concat(holder)
    # print(data)
    return df
data = stock_list(stocks)
data

C:\Users\lklun\AppData\Local\Temp\ipykernel_20080\1661971068.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,148.816,44232842245,N/A


In [4]:
def chunks(lst, n):
    
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [5]:
def batch_call(value):
    symbols = list(chunks(stocks['Ticker'], 100))
    symbol_strings = []
    
    for i in range(0, len(symbols)):
        symbol_strings.append(','.join(symbols[i]))
    
    df = pd.DataFrame(columns=my_columns)
    for name in symbol_strings:
        
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={name}&types=quote&token={IEX_CLOUD_API_TOKEN}'
        datas = requests.get(batch_api_call_url).json()
        
        for quote in name.split(','):
            try:
                df = df.append(
                pd.Series(
                    [
                        quote,
                        datas[quote]['quote']['latestPrice'],
                        datas[quote]['quote']['marketCap'],
                        'N/A'   
                    ],
                index=my_columns),
                ignore_index = True
                )
            except:
                # print(quote) 
                continue
    return df

dataset = batch_call(stocks)
dataset

C:\Users\lklun\AppData\Local\Temp\ipykernel_20080\3240815737.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\lklun\AppData\Local\Temp\ipykernel_20080\3240815737.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\lklun\AppData\Local\Temp\ipykernel_20080\3240815737.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\lklun\AppData\Local\Temp\ipykernel_20080\3240815737.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\lklun\AppData\Local\Temp\ipykernel_20080\3240815737.py:16: FutureWarning: The frame.append method is deprecated and

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,150.7,44034128203,N/A
1,AAL,14.5,9303917701,N/A
2,AAP,158.097,9280663178,N/A
3,AAPL,154.88,2412877056570,N/A
4,ABBV,159.82,272744679534,N/A
...,...,...,...,...
496,YUM,126.41,35915876234,N/A
497,ZBH,120.23,25273869871,N/A
498,ZBRA,269.1,13809337207,N/A
499,ZION,53.23,7964336291,N/A


In [6]:
# portfolio_size = input('portfolio size??')
portfolio_size = 1000000

try:
    val = float(portfolio_size)
except:
    print("That's not a number \nPlease Try again")
    portfolio_size = input('portfolio size??')
    val = float(portfolio_size)


position_size = val/len(dataset.index)

for i in range(len(dataset.index)):
    dataset.loc[i, "Number of Shares to buy"] = math.floor(position_size/dataset.loc[i, 'Stock Price'])
    
dataset

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,150.7,44034128203,13
1,AAL,14.5,9303917701,137
2,AAP,158.097,9280663178,12
3,AAPL,154.88,2412877056570,12
4,ABBV,159.82,272744679534,12
...,...,...,...,...
496,YUM,126.41,35915876234,15
497,ZBH,120.23,25273869871,16
498,ZBRA,269.1,13809337207,7
499,ZION,53.23,7964336291,37


In [7]:
dataset.to_excel('recommended_trades.xlsx')